# Capstone Project
### this notebook will be mainly used for the capstone project.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
wiki_link="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page=requests.get(wiki_link).text
soup = BeautifulSoup(page,'xml')
#soup

# Extract data from table


In [3]:
tables = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []



## Reading table data

In [4]:
for table in tables.find_all('tr'):
    count=1
    Borough_var=-1
    postcode_var=-1
    Neighbor_var=-1
    t= table.find_all('td')
    for i in t:
        if count==1:
            postcode_var=i.text
        if count==2:
            Borough_var=i.text
            tag_borough=i.find('a')
        if count==3:
            Neighbor_var=str(i.text).strip()
            tag_neighbor=i.find('a')
        
        count+=1
        #print(t)
    if (Borough_var == 'Not assigned'):
        continue
    if (Neighbor_var=='Not assigned') and (Borough_var != 'Not assigned'):
        Neighbor_var=Borough_var
    try:
        if (tag_borough is None):
            continue
    except:
        pass

    
    Postcode.append(postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbor_var)
                         
        

## Converting to table dataframe

In [5]:
l=len(Postcode)
m=set(Postcode)
print(len(m))

Post_u      = []
Bor_u       = []
Neighbour_u = []


for post in m:
    p_var = ''; b_var = ''; n_var = ''; 
    for post_id, n in enumerate(Postcode):
        if post == n:
            p_var = post;
            b_var = Borough[post_id]
            if n_var == '': 
                n_var = Neighbourhood[post_id]
            else:
                n_var = n_var + ', ' + Neighbourhood[post_id]
    Post_u.append(p_var)
    Bor_u.append(b_var)
    Neighbour_u.append(n_var)

101


In [6]:
toronto={'Postcode':Post_u,'Borough': Bor_u,'Neighbourhood':Neighbour_u}
df=pd.DataFrame.from_dict(toronto)

# Add Longitude and lattitude

In [7]:
import io
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.columns = ['Postcode', 'Latitude', 'Longitude']
df = pd.merge(c, df, on='Postcode')
df = df[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]


In [8]:
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [9]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Find co-ordinates for Toronto

In [10]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [11]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [12]:
Scar_data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
Scar_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Selected Scarborough as a location to search nearby area

In [13]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="Tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [14]:
map_scar = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Scar_data['Latitude'], Scar_data['Longitude'], Scar_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#aaddcc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scar)  
    
map_scar

In [15]:
CLIENT_ID = 'T0NU0BAWO1FPR2GHELUOTY2G50YRLESBYB54LW5GI2XBJFEB' # your Foursquare ID
CLIENT_SECRET = '1VB0E40B4L5SZXRWXY1F0RMLIQOW5TXMN0KXO1TDARVMHWX5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T0NU0BAWO1FPR2GHELUOTY2G50YRLESBYB54LW5GI2XBJFEB
CLIENT_SECRET:1VB0E40B4L5SZXRWXY1F0RMLIQOW5TXMN0KXO1TDARVMHWX5


In [16]:
Scar_data.loc[0, 'Neighbourhood']

'Rouge, Malvern'

In [17]:
neighborhood_latitude = Scar_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Scar_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Scar_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


In [18]:
LIMIT=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=T0NU0BAWO1FPR2GHELUOTY2G50YRLESBYB54LW5GI2XBJFEB&client_secret=1VB0E40B4L5SZXRWXY1F0RMLIQOW5TXMN0KXO1TDARVMHWX5&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [19]:
results = requests.get(url).json()
#results

## Explore nearby areas using foursquare api

In [20]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()
nearby_venues.shape

(2, 4)

In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
scar_venues = getNearbyVenues(names=Scar_data['Neighbourhood'],
                                   latitudes=Scar_data['Latitude'],
                                   longitudes=Scar_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


In [25]:
print(scar_venues.shape)
scar_venues.head()

(1684, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Toronto Pan Am Sports Centre,43.790623,-79.193869,Athletics & Sports
1,"Rouge, Malvern",43.806686,-79.194353,African Rainforest Pavilion,43.817725,-79.183433,Zoo Exhibit
2,"Rouge, Malvern",43.806686,-79.194353,Toronto Zoo,43.820582,-79.181551,Zoo
3,"Rouge, Malvern",43.806686,-79.194353,Polar Bear Exhibit,43.823372,-79.185145,Zoo
4,"Rouge, Malvern",43.806686,-79.194353,Australasia Pavillion,43.822563,-79.183286,Zoo Exhibit


In [26]:
scar_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,100,100,100,100,100,100
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",100,100,100,100,100,100
"Birch Cliff, Cliffside West",100,100,100,100,100,100
Cedarbrae,100,100,100,100,100,100
"Clairlea, Golden Mile, Oakridge",100,100,100,100,100,100
"Clarks Corners, Sullivan, Tam O'Shanter",100,100,100,100,100,100
"Cliffcrest, Cliffside, Scarborough Village West",100,100,100,100,100,100
"Dorset Park, Scarborough Town Centre, Wexford Heights",100,100,100,100,100,100
"East Birchmount Park, Ionview, Kennedy Park",100,100,100,100,100,100


In [27]:
# one hot encoding
scar_onehot = pd.get_dummies(scar_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scar_onehot['Neighbourhood'] = scar_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])
scar_onehot = scar_onehot[fixed_columns]

scar_onehot.head()

,Neighbourhood,Afghan Restaurant,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,...,Toy / Game Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Zoo,Zoo Exhibit
0,"Rouge, Malvern",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [28]:
scar_grouped = scar_onehot.groupby('Neighbourhood').mean().reset_index()
scar_grouped.head()

,Neighbourhood,Afghan Restaurant,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,...,Toy / Game Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Zoo,Zoo Exhibit
0,Agincourt,0.00,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,...,0.01,0.0,0.00,0.01,0.0,0.01,0.00,0.0,0.0,0.0
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.01,0.01,0.02,0.00,0.01,0.01,0.00,0.00,...,0.00,0.0,0.00,0.02,0.0,0.03,0.00,0.0,0.0,0.0
2,"Birch Cliff, Cliffside West",0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.01,...,0.01,0.0,0.01,0.02,0.0,0.00,0.01,0.0,0.0,0.0
3,Cedarbrae,0.00,0.01,0.01,0.02,0.01,0.00,0.00,0.01,0.00,...,0.01,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0
4,"Clairlea, Golden Mile, Oakridge",0.01,0.02,0.00,0.01,0.00,0.00,0.01,0.00,0.01,...,0.01,0.0,0.02,0.01,0.0,0.01,0.01,0.0,0.0,0.0


In [29]:
scar_grouped.shape

(17, 137)

In [30]:
num_top_venues = 5

for hood in scar_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = scar_grouped[scar_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.13
1           Coffee Shop  0.06
2     Indian Restaurant  0.05
3  Caribbean Restaurant  0.05
4      Sushi Restaurant  0.04


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                  venue  freq
0    Chinese Restaurant  0.13
1  Caribbean Restaurant  0.06
2                Bakery  0.05
3       Bubble Tea Shop  0.04
4     Indian Restaurant  0.04


----Birch Cliff, Cliffside West----
            venue  freq
0           Beach  0.08
1            Park  0.07
2     Coffee Shop  0.07
3            Café  0.05
4  Breakfast Spot  0.05


----Cedarbrae----
                  venue  freq
0           Coffee Shop  0.11
1     Indian Restaurant  0.06
2    Chinese Restaurant  0.06
3        Sandwich Place  0.06
4  Fast Food Restaurant  0.06


----Clairlea, Golden Mile, Oakridge----
                       venue  freq
0  Middle Eastern Restaurant  0.06
1                       Park  0.06
2                Coffee Sh

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = scar_grouped['Neighbourhood']

for ind in np.arange(scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Coffee Shop,Indian Restaurant,Caribbean Restaurant,Sushi Restaurant,Park,Hakka Restaurant,Noodle House,Supermarket,Bakery
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Caribbean Restaurant,Bakery,Bubble Tea Shop,Indian Restaurant,Japanese Restaurant,Breakfast Spot,Vietnamese Restaurant,Noodle House,Supermarket
2,"Birch Cliff, Cliffside West",Beach,Park,Coffee Shop,Breakfast Spot,Café,Gastropub,Fish & Chips Shop,Bakery,Pub,Burger Joint
3,Cedarbrae,Coffee Shop,Indian Restaurant,Sandwich Place,Chinese Restaurant,Fast Food Restaurant,Caribbean Restaurant,Pizza Place,Park,Pharmacy,Breakfast Spot
4,"Clairlea, Golden Mile, Oakridge",Middle Eastern Restaurant,Park,Coffee Shop,Gastropub,Beach,Breakfast Spot,Gym / Fitness Center,Café,Supermarket,Pub


In [33]:
kclusters = 5

scar_grouped_clustering = scar_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 3, 1, 3, 2, 1, 4, 4, 1], dtype=int32)

In [34]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scar_merged = Scar_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scar_merged = scar_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
#scar_merged['Cluster Labels'] = scar_merged['Cluster Labels'].astype(int)
scar_merged.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Zoo Exhibit,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Fast Food Restaurant,Breakfast Spot,Grocery Store,Park,Chinese Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Zoo Exhibit,Pizza Place,Coffee Shop,Park,Breakfast Spot,Pharmacy,Beer Store,Discount Store,Pub,Smoothie Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Beer Store,Bank,Fast Food Restaurant,Park,Indian Restaurant,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Fast Food Restaurant,Pizza Place,Pharmacy,Park,Sandwich Place,Breakfast Spot,Burger Joint,Caribbean Restaurant,Grocery Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Coffee Shop,Indian Restaurant,Sandwich Place,Chinese Restaurant,Fast Food Restaurant,Caribbean Restaurant,Pizza Place,Park,Pharmacy,Breakfast Spot
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Coffee Shop,Burger Joint,Park,Chinese Restaurant,Indian Restaurant,Fast Food Restaurant,Breakfast Spot,Sandwich Place,Supermarket,Sushi Restaurant
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,4,Coffee Shop,Park,Middle Eastern Restaurant,Chinese Restaurant,Burger Joint,Indian Restaurant,Supermarket,Burrito Place,Grocery Store,Gym
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,3,Middle Eastern Restaurant,Park,Coffee Shop,Gastropub,Beach,Breakfast Spot,Gym / Fitness Center,Café,Supermarket,Pub
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,1,Coffee Shop,Sandwich Place,Park,Burger Joint,Pharmacy,Chinese Restaurant,Grocery Store,Bank,Pizza Place,Beer Store
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,3,Beach,Park,Coffee Shop,Breakfast Spot,Café,Gastropub,Fish & Chips Shop,Bakery,Pub,Burger Joint


In [35]:
scar_merged=scar_merged.dropna()
scar_merged.shape
scar_merged['Cluster Labels'].unique()

array([0, 1, 4, 3, 2])

In [36]:
scar_merged['Cluster Labels']=scar_merged['Cluster Labels'].astype(int)

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scar_merged['Latitude'], scar_merged['Longitude'], scar_merged['Neighbourhood'], scar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
scar_merged.loc[scar_merged['Cluster Labels'] == 0, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Zoo Exhibit,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Fast Food Restaurant,Breakfast Spot,Grocery Store,Park,Chinese Restaurant
1,Scarborough,0,Zoo Exhibit,Pizza Place,Coffee Shop,Park,Breakfast Spot,Pharmacy,Beer Store,Discount Store,Pub,Smoothie Shop
16,Scarborough,0,Zoo Exhibit,Sandwich Place,Fast Food Restaurant,Pizza Place,Coffee Shop,Pharmacy,Grocery Store,Restaurant,Hakka Restaurant,Supermarket


In [39]:
scar_merged.loc[scar_merged['Cluster Labels'] == 1, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Beer Store,Bank,Fast Food Restaurant,Park,Indian Restaurant,Fried Chicken Joint
3,Scarborough,1,Coffee Shop,Fast Food Restaurant,Pizza Place,Pharmacy,Park,Sandwich Place,Breakfast Spot,Burger Joint,Caribbean Restaurant,Grocery Store
4,Scarborough,1,Coffee Shop,Indian Restaurant,Sandwich Place,Chinese Restaurant,Fast Food Restaurant,Caribbean Restaurant,Pizza Place,Park,Pharmacy,Breakfast Spot
5,Scarborough,1,Coffee Shop,Burger Joint,Park,Chinese Restaurant,Indian Restaurant,Fast Food Restaurant,Breakfast Spot,Sandwich Place,Supermarket,Sushi Restaurant
8,Scarborough,1,Coffee Shop,Sandwich Place,Park,Burger Joint,Pharmacy,Chinese Restaurant,Grocery Store,Bank,Pizza Place,Beer Store


In [40]:
scar_merged.loc[scar_merged['Cluster Labels'] == 2, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,2,Chinese Restaurant,Coffee Shop,Indian Restaurant,Caribbean Restaurant,Sushi Restaurant,Park,Hakka Restaurant,Noodle House,Supermarket,Bakery
13,Scarborough,2,Chinese Restaurant,Caribbean Restaurant,Coffee Shop,Middle Eastern Restaurant,Supermarket,Indian Restaurant,Burger Joint,Japanese Restaurant,Grocery Store,Sushi Restaurant
14,Scarborough,2,Chinese Restaurant,Caribbean Restaurant,Bakery,Bubble Tea Shop,Indian Restaurant,Japanese Restaurant,Breakfast Spot,Vietnamese Restaurant,Noodle House,Supermarket
15,Scarborough,2,Chinese Restaurant,Caribbean Restaurant,Bubble Tea Shop,Japanese Restaurant,Bakery,Sushi Restaurant,Asian Restaurant,Korean Restaurant,Burger Joint,Burrito Place


In [41]:
scar_merged.loc[scar_merged['Cluster Labels'] == 3, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough,3,Middle Eastern Restaurant,Park,Coffee Shop,Gastropub,Beach,Breakfast Spot,Gym / Fitness Center,Café,Supermarket,Pub
9,Scarborough,3,Beach,Park,Coffee Shop,Breakfast Spot,Café,Gastropub,Fish & Chips Shop,Bakery,Pub,Burger Joint


In [42]:
scar_merged.loc[scar_merged['Cluster Labels'] == 4, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,4,Coffee Shop,Park,Middle Eastern Restaurant,Chinese Restaurant,Burger Joint,Indian Restaurant,Supermarket,Burrito Place,Grocery Store,Gym
10,Scarborough,4,Coffee Shop,Middle Eastern Restaurant,Caribbean Restaurant,Indian Restaurant,Chinese Restaurant,Clothing Store,Supermarket,Sushi Restaurant,Gym,Grocery Store
11,Scarborough,4,Middle Eastern Restaurant,Coffee Shop,Supermarket,Burger Joint,Indian Restaurant,Caribbean Restaurant,Chinese Restaurant,Mediterranean Restaurant,Discount Store,Pet Store


## five types of locations have been found

#### A user can use these clusters to find out the kind of places one wants to be around with.
If we  get more data more exclusive clusters of location can be found out.


Thanks